In [ ]:
from functions import *
import numpy as np
from scipy.integrate import quad
import matplotlib.pyplot as plt
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KernelDensity
from tqdm import tqdm
import math
import cmath
from scipy.integrate import simpson, romb
from scipy.stats import iqr, norm, gaussian_kde
from scipy.special import erfc, wofz
import pickle
import os

In [ ]:
theta_list, x_list = np.loadtxt("DataPhase28.dat", unpack=True)
T = len(theta_list)
alpha = 0.3
r2 = 0.925

L_limit = 4
n_precision = 10000
X_precision = np.linspace(-L_limit, L_limit, n_precision)
N_theta = 20
N_bins = 30

gmm_components = 2

In [ ]:
grouped_theta_list, grouped_data_dict = data_wrapper(theta_list, x_list, N_theta)

In [ ]:
trial_bin = 1

xb = np.array(grouped_data_dict[trial_bin])
W_Gaussian = reconstruct_pdf_mle(xb, X_precision, gmm_components)


hist_counts, bin_edges = np.histogram(grouped_data_dict[trial_bin], bins=N_bins, density=True)
bin_centers = 0.5 * (bin_edges[:-1] + bin_edges[1:])
plt.bar(bin_centers, hist_counts, width=(bin_edges[1] - bin_edges[0]), alpha=0.6, label="Experimental Data", color="gray", edgecolor="black")
plt.plot(X_precision, P_x_theta(X_precision, grouped_theta_list[trial_bin], alpha, r2), label="Theoretical PDF", color="red", linewidth=2)
plt.plot(X_precision, W_Gaussian, color = 'blue', label="GMM KLE")


plt.legend()
plt.xlabel(r"$x$")
plt.ylabel(r"$p(x|\theta)$")

plt.title(f"Tomogram, $θ$ = {grouped_theta_list[trial_bin]:.2f}")
plt.xlim(-L_limit, L_limit)

In [ ]:
gmm_components = 1
error = []
for trial_bin in range(N_theta):
    xb = np.array(grouped_data_dict[trial_bin])
    W_Gaussian = reconstruct_pdf_mle(xb, X_precision, gmm_components)
    P_theoretical = P_x_theta(X_precision, grouped_theta_list[trial_bin], alpha, r2)
    error_step = simpson(y = (P_theoretical - W_Gaussian)**2, x = X_precision)
    error.append(error_step)

np.mean(error)

In [ ]:
# Generate the pdf plot

# Read W_Gaussian from pickle
with open("W_Gaussian.pkl", "rb") as f:
    KDE_W_Gaussian = pickle.load(f)


trial_bin = 1



xb = np.array(grouped_data_dict[trial_bin])

hist_counts, bin_edges = np.histogram(grouped_data_dict[trial_bin], bins=N_bins, density=True)
bin_centers = 0.5 * (bin_edges[:-1] + bin_edges[1:])
plt.bar(bin_centers, hist_counts, width=(bin_edges[1] - bin_edges[0]), alpha=0.6, label="Data Histogram", color="gray", edgecolor="black")

theorical_pdf = P_x_theta(X_precision, grouped_theta_list[trial_bin], alpha, r2)
plt.plot(X_precision, theorical_pdf, label="Theoretical PDF", color="red", linewidth=2)
plt.plot(X_precision, KDE_W_Gaussian, color = 'green', label="Gaussian KDE", linewidth=2)  # Added line to plot KDE

MSE = simpson(y = (theorical_pdf - KDE_W_Gaussian)**2, x = X_precision)
print("MSE Gaussian KDE:", MSE)


gmm_components = 2
W_Gaussian = reconstruct_pdf_mle(xb, X_precision, gmm_components)
plt.plot(X_precision, W_Gaussian, color = 'blue', label="MLE ($GMM=$" + str(gmm_components) + ")", linewidth=2)

MSE = simpson(y = (theorical_pdf - W_Gaussian)**2, x = X_precision)
print("MSE gmm2:", MSE)


gmm_components = 1
W_Gaussian = reconstruct_pdf_mle(xb, X_precision, gmm_components)
plt.plot(X_precision, W_Gaussian, color = 'orange', label="MLE ($GMM=$" + str(gmm_components) + ")", linewidth=2)

MSE = simpson(y = (theorical_pdf - W_Gaussian)**2, x = X_precision)
print("MSE gmm1:", MSE)



plt.legend()
plt.xlabel(r"$X$")
plt.ylabel(r"$\hat{\mathcal{W}}(X)$")

plt.title(r"$\hat{\mathcal{W}}(X)$ Estimation for $n\approx$" + str(int(T/N_theta)) + r", $\theta\approx$"+str(np.round(grouped_theta_list[trial_bin],2)))
plt.xlim(-L_limit, L_limit)
plt.grid(alpha=0.3)

plt.tight_layout()
plt.savefig("lvovsky_pdf.pdf", dpi=300)